In [1]:
# Carrega as variáveis de embiente usadas pelo notebook. Estas variáveis devem ser criadas no arquivo .env, localizada no mesmo diretório do notebook
# As seguintes variáveis devem ser criadas:
# - export STR_CON=<string de conexão com o banco de dados relacional>
# - export RAW_DATA_PATH=<caminho onde os arquivos contendo o histórico de preço de cada ativo será armazenado>
# - export DATASET_PATH=<caminho onde os arquivos contendo o histórico de preço e os indicadores técnicos serão armazenados>
# - export MODELS_PATH=<caminnho onde os modelos treinados serão armazenados>
# - export TRAIN_DATASET=<caminho onde os datasets estandarizados e formatados para o treinamento serão armazenados>
%load_ext dotenv
%dotenv

# Garante que os scripts Python sejam recarregados em cada execução do notebook.
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
import os
import pandas as pd
import numpy as np
import talib as ta
from data_preparation import PreProcess, PivotLevels
from db_access import StockHistory, ExportToParquet, ExportToPickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

pd.options.display.max_columns = 200

## Preparacao dos datasets

O código abaixo utiliza os arquivos parquet previamente extraídos do banco de dados.

In [69]:
pre_process = PreProcess()
parquet_exporter = ExportToParquet()
pickle_exporter = ExportToPickle()

raw_file_path = os.environ.get("RAW_DATA_PATH")
data_file_path = os.environ.get("DATASET_PATH")
model_base_path = os.environ.get("MODELS_PATH")
str_conn = os.environ.get("STR_CON")
stride_coef_ang = 1

# Colunas que serão usadas MinMaxScaler
columns_to_scale = ['close', 'rsi', 'macd', 'macdsignal', 'macdhist', 'slowk', 'slowd', 'willr', 'mfi',
                    'coef_ang_volume', 'var_volume', 'diff_volume_vol_med',
                    'diff_close_r1', 'diff_close_r2', 'diff_close_r3', 'diff_close_s1', 'diff_close_s2', 'diff_close_s3',
                    'diff_willr_20', 'diff_willr_80', 'dif_mfi_20', 'diff_mfi_80', 
                    'diff_rsi_70', 'diff_k_d']
# Colunas com sinal de negociação. O dataframe usado pelo modelo terá uma série temporal e colunas com sinal de negociação. A coluna com a série temporal
# será passada para autoencoder, que irá gerar um vetor que representa a série, este vetor será concatenado com as colunas de sinal e passarão por uma MLP
# que fará a classificação do ativo
signal_cols = ['DR3', 'M2', 'MA', 'MB', 'N/D', 'N1', 'N2', 'NM', 
               'CDLMORNINGSTAR', 'CDLSHOOTINGSTAR', 'CDLTAKURI', 'CDLHANGINGMAN', 'CDLEVENINGDOJISTAR', 'CDL3OUTSIDE', 'CDLONNECK', 'CDLHAMMER', 'CDLHARAMICROSS',
               'diff_volume_vol_med', 'diff_close_r1', 'diff_close_r2', 'diff_close_r3', 'diff_close_s1',
               'diff_close_s2', 'diff_close_s3', 'coef_ang_volume', 'var_volume',
               'diff_willr_20', 'diff_willr_80', 'dif_mfi_20', 'diff_mfi_80', 'diff_rsi_70', 'diff_k_d',
               'volatiliade', 'coef_ang_close', 'coef_ang_volat', 'coef_ang_macdhist', 'coef_ang_rsi', 'coef_ang_mfi']
# Colunas que não precisam passar pelo MinMaxScaler
columns_to_pop = ['ticker', 'DR3', 'M2', 'MA', 'MB', 'N/D', 'N1', 'N2', 'NM', 
                  'CDLMORNINGSTAR', 'CDLSHOOTINGSTAR', 'CDLTAKURI', 'CDLHANGINGMAN', 'CDLEVENINGDOJISTAR', 'CDL3OUTSIDE', 'CDLONNECK', 'CDLHAMMER', 'CDLHARAMICROSS',
                  'volatiliade', 'coef_ang_close', 'coef_ang_volat', 'coef_ang_macdhist', 'coef_ang_rsi', 'coef_ang_mfi']

path_content = os.listdir(raw_file_path)
# Filtra os arquivos parquet do diretório
path_content = [file for file in path_content if file.endswith(".parquet")]

# Faz o one hot encoder do mercado onde o emissor do ativo é negociado
dbo = StockHistory(str_conn)
markets = dbo.select_markets()
one_hot = OneHotEncoder(sparse=False)
one_hot.fit(markets.values)

for file in path_content:
    print(f"Processando arquivo: {raw_file_path}/{file}")
    df_raw = pd.read_parquet(os.path.join(raw_file_path, file))
    market_one_hot = pd.DataFrame(one_hot.transform(df_raw['market'].values.reshape(-1,1)), columns = one_hot.categories_[0])
    df_raw = pd.concat([df_raw.reset_index(), market_one_hot], axis=1).set_index('index')
    df_raw.set_index("dt_price", drop=False, inplace=True)

    # Calcula os indicadores técnicos definidos nas estratégias para cada ativo
    for strategy_name, strategy, df_tech in pre_process.calculate_strategy("strategies.json", df_raw):
        window_size = strategy["historic_period"]
        stride = strategy["stride"]
        profit_period = strategy["profit_period"]
        profit = strategy["profit"]

        # Ajusta d valor do ROC
        df_tech['roc'] = df_tech['roc']/100
        # Calcula a volatilidade do ativo no periodo window_size
        df_tech['volatiliade'] = df_tech['roc'].rolling(window=window_size).std()
        # Calcula a variacao do volume
        df_tech['var_volume'] = ta.ROC(df_tech['volume'], 1)/100
        df_tech[f'vol_med_{window_size}_days'] = df_tech['volume'].rolling(window=window_size).mean()
        df_tech['diff_volume_vol_med'] = df_tech['volume']/df_tech[f'vol_med_{window_size}_days']-1

        # Calcula os Pivot Levels
        pivot_levels = PivotLevels(high=df_tech['high'], low=df_tech['low'], close=df_tech['close'])

        # Diferenca entre os Pivot Levels e o preco de fechamento
        df_tech['diff_close_r1'] = df_tech['close'] - pivot_levels.R1
        df_tech['diff_close_r2'] = df_tech['close'] - pivot_levels.R2
        df_tech['diff_close_r3'] = df_tech['close'] - pivot_levels.R3
        df_tech['diff_close_s1'] = df_tech['close'] - pivot_levels.S1
        df_tech['diff_close_s2'] = df_tech['close'] - pivot_levels.S2
        df_tech['diff_close_s3'] = df_tech['close'] - pivot_levels.S3

        # Caclula o coeficiente angular da regressao do preco de fechamento e adiciona ao dataframe de treinamento
        df_tech['coef_ang_close'] = pre_process.linear_regression_slope(df_tech['close'], window_size=window_size, stride=stride_coef_ang)
        df_tech['coef_ang_volume'] = pre_process.linear_regression_slope(df_tech['volume'], window_size=window_size, stride=stride_coef_ang)
        # Remove os valores nulos do DataFrame
        df_tech.dropna(inplace=True)

        df_tech['coef_ang_volat'] = pre_process.linear_regression_slope(df_tech['volatiliade']*100, window_size=window_size, stride=stride_coef_ang)
        df_tech['coef_ang_macdhist'] = pre_process.linear_regression_slope(df_tech['macdhist'], window_size=window_size, stride=stride_coef_ang)
        df_tech['coef_ang_rsi'] = pre_process.linear_regression_slope(df_tech['rsi'], window_size=window_size, stride=stride_coef_ang)
        df_tech['coef_ang_mfi'] = pre_process.linear_regression_slope(df_tech['mfi'], window_size=window_size, stride=stride_coef_ang)

        # Remove os valores nulos do DataFrame
        df_tech.dropna(inplace=True)

        # Cria um dataframe auxilizar para armazenar as colunas que serao usadas no modelo, mas nao precisam ser redimensionadas 
        df_aux = pd.DataFrame()
        for col in columns_to_pop:
            df_aux[col] = df_tech.pop(col)

        scaler = MinMaxScaler().fit(df_tech[columns_to_scale])
        transformed_data = scaler.transform(df_tech[columns_to_scale])
        df_tech = pd.concat([df_aux.reset_index(), pd.DataFrame(transformed_data, columns=scaler.get_feature_names_out())], axis=1).set_index('dt_price', drop=False)

        # Formata o dataset para ser usado no modelo
        df_model = pre_process.format_dataset(df_raw, df_tech, window_size, stride, profit_period, profit, None, signal_cols)

        # Exporta o data set como um arquivo parquet
        parquet_exporter.export(df_model, os.path.join(data_file_path, strategy_name), file.split('.')[0])

Processando arquivo: data/raw/CPFE3.parquet
Processing strategy: Modelo para selecionar ativos atraves de sinais de negociacao
Calculating MACD...
Calculating RSI...
Calculating STOCH...
Calculating WILLR_14...
Calculating MFI_14...
Calculating ROC...
Processando arquivo: data/raw/BEEF3.parquet
Processing strategy: Modelo para selecionar ativos atraves de sinais de negociacao
Calculating MACD...
Calculating RSI...
Calculating STOCH...
Calculating WILLR_14...
Calculating MFI_14...
Calculating ROC...
Processando arquivo: data/raw/LOGG3.parquet
Processing strategy: Modelo para selecionar ativos atraves de sinais de negociacao
Calculating MACD...
Calculating RSI...
Calculating STOCH...
Calculating WILLR_14...
Calculating MFI_14...
Calculating ROC...
Processando arquivo: data/raw/VIVT3.parquet
Processing strategy: Modelo para selecionar ativos atraves de sinais de negociacao
Calculating MACD...
Calculating RSI...
Calculating STOCH...
Calculating WILLR_14...
Calculating MFI_14...
Calculating

In [67]:
df_tech[signal_cols].shape, len(signal_cols), df_tech.shape
# [*df_tech[['DR3', 'M2', 'MA', 'MB', 'N/D', 'N1', 'N2', 'NM']].iloc[5].values]
df_tech[signal_cols].head()

,DR3,M2,MA,MB,N/D,N1,N2,NM,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLTAKURI,CDLHANGINGMAN,CDLEVENINGDOJISTAR,CDL3OUTSIDE,CDLONNECK,CDLHAMMER,CDLHARAMICROSS,diff_volume_vol_med,diff_volume_vol_med,diff_close_r1,diff_close_r1,diff_close_r2,diff_close_r2,diff_close_r3,diff_close_r3,diff_close_s1,diff_close_s1,diff_close_s2,diff_close_s2,diff_close_s3,diff_close_s3,coef_ang_volume,var_volume,diff_volume_vol_med,diff_volume_vol_med,diff_close_r1,diff_close_r1,diff_close_r2,diff_close_r2,diff_close_r3,diff_close_r3,diff_close_s1,diff_close_s1,diff_close_s2,diff_close_s2,diff_close_s3,diff_close_s3,diff_willr_20,diff_willr_80,dif_mfi_20,diff_mfi_80,diff_rsi_70,diff_k_d,volatiliade,coef_ang_close,coef_ang_volat,coef_ang_macdhist,coef_ang_rsi,coef_ang_mfi
dt_price,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-04-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.047492,0.047492,0.937132,0.937132,0.937132,0.937132,0.934413,0.934413,0.077519,0.077519,0.077519,0.077519,0.072221,0.072221,0.424839,0.079671,0.047492,0.047492,0.937132,0.937132,0.937132,0.937132,0.934413,0.934413,0.077519,0.077519,0.077519,0.077519,0.072221,0.072221,0.490385,0.490385,0.478947,0.478947,0.620362,0.390486,0.014446,0.029494,-0.002067,-0.002777,0.216311,0.219151
2013-04-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.073078,0.073078,0.948919,0.948919,0.948919,0.948919,0.932062,0.932062,0.135659,0.135659,0.135659,0.135659,0.109080,0.109080,0.425581,0.109064,0.073078,0.073078,0.948919,0.948919,0.948919,0.948919,0.932062,0.932062,0.135659,0.135659,0.135659,0.135659,0.109080,0.109080,0.927835,0.927835,0.481106,0.481106,0.709610,0.536827,0.014539,0.029103,-0.002676,-0.003018,0.204741,0.094743
2013-05-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.063097,0.063097,0.953831,0.953831,0.953831,0.953831,0.946433,0.946433,0.083979,0.083979,0.083979,0.083979,0.071621,0.071621,0.424665,0.061959,0.063097,0.063097,0.953831,0.953831,0.953831,0.953831,0.946433,0.946433,0.083979,0.083979,0.083979,0.083979,0.071621,0.071621,0.913793,0.913793,0.560840,0.560840,0.740219,0.681148,0.014452,0.029667,-0.003246,-0.003082,0.212400,0.011943
2013-05-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.080204,0.080204,0.950884,0.950884,0.950884,0.950884,0.950353,0.950353,0.052972,0.052972,0.052972,0.052972,0.050944,0.050944,0.425880,0.091382,0.080204,0.080204,0.950884,0.950884,0.950884,0.950884,0.950353,0.950353,0.052972,0.052972,0.052972,0.052972,0.050944,0.050944,0.842520,0.842520,0.664319,0.664319,0.742129,0.717653,0.014053,0.029034,-0.003870,-0.003101,0.202363,-0.047147
2013-05-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.237148,0.237148,0.960707,0.960707,0.960707,0.960707,0.958714,0.958714,0.050388,0.050388,0.050388,0.050388,0.046149,0.046149,0.448219,0.224658,0.237148,0.237148,0.960707,0.960707,0.960707,0.960707,0.958714,0.958714,0.050388,0.050388,0.050388,0.050388,0.046149,0.046149,0.795082,0.795082,0.528376,0.528376,0.723491,0.524019,0.013778,0.029047,-0.004435,-0.003042,0.204103,-0.120955
